In [87]:
import numpy as np
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import ipaddress
es_client = Elasticsearch(http_compress=True)

In [88]:
import random
from random import randrange

def generateIP():
    blockOne = randrange(0, 255, 1)
    blockTwo = randrange(0, 255, 1)
    blockThree = randrange(0, 255, 1)
    blockFour = randrange(0, 255, 1)

    return str(blockOne) + '.' + str(blockTwo) + '.' + str(blockThree) + '.' + str(blockFour)

In [89]:
%time
def generate(median=630, err=12, outlier_err=400, size=30000, outlier_size=1500):
    flow_errs = err * np.random.rand(size) * np.random.choice((-1, 1), size)
    flow_good = median + flow_errs
    sessions_good = np.random.rand(size) * 10 + 1
    sessions_good = np.round(sessions_good,0)
    combined_good = np.vstack((flow_good, sessions_good)).T 


    flow_lower_errs = outlier_err * np.random.rand(outlier_size)
    flow_lower_outliers = median - err -flow_lower_errs
    sessions_lower_outliers = np.random.rand(outlier_size) * 1 + 1
    sessions_lower_outliers = np.round(sessions_lower_outliers,0)
    combined_lower = np.vstack((flow_lower_outliers, sessions_lower_outliers)).T

    flow_upper_errs = outlier_err * np.random.rand(outlier_size)
    flow_upper_outliers = median + err + flow_upper_errs
    sessions_upper_outliers = np.random.rand(outlier_size) * 1000
    sessions_upper_outliers = np.round(sessions_upper_outliers,0)
    combined_upper = np.vstack((flow_upper_outliers, sessions_upper_outliers)).T

    data = np.concatenate((combined_good, combined_lower, combined_upper))
    #   np.random.shuffle(data)

    return data

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [90]:
data = generate()
df = pd.DataFrame(data=generate())
# "nb_sessions", "nb_src_ip", "nb_dest_ip", "sum_url", "sum_bytes_toclient", "sum_bytes_toserver", "ratio_server_client"

In [91]:
df['source_ip'] = '190.166.13.79'
for i in df.index:
    if i < 30000:
        df.loc[i, 'source_ip'] = '192.11.13.'+ str(random.randrange(150, 180))
    else:
        df.loc[i, 'source_ip'] = generateIP()

In [92]:
df['outliers'] = 0
df[30000:]['outliers'] = 1

In [93]:
df = df.rename(columns = {0 : "sum_bytes_toclient", 1 : "nb_sessions"})
df['sum_bytes_toclient'] = df['sum_bytes_toclient'].round(0)
df = df.astype({'sum_bytes_toclient': int, 'nb_sessions': int, 'source_ip': str})

In [94]:
df['ts'] = pd.date_range("2021-01-27 9:28:00", periods=32400, freq="min")
df['id'] = np.linspace(1, 32400, 32400)

In [95]:
use_these_keys = ['sum_bytes_toclient', 'nb_sessions', 'source_ip', 'outliers','ts','id']
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

In [96]:
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": 'anomaly_example',
                "_type": "_doc",
                "_id" : f"{document['id']}",
                "_source": filterKeys(document),
            }
    raise StopIteration
helpers.bulk(es_client, doc_generator(df))

(32400, [])